<a href="https://colab.research.google.com/github/lovepreetmultani/Anti-Money-Laundering/blob/main/Tiger_graph_laundering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pyTigerGraph
import pyTigerGraph as tg
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 6.9 MB/s eta 0:00:00


In [4]:
# Connection parameters
hostName = "https://5d8cb4af4a294e9c852fad8b03bddd05.i.tgcloud.io"
userName = "user_1"
password = "Algo@123"
graphName="test_graph"
secret="q35on53p9dhtev14otqt0223u5k194a0"
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")


Connected


In [5]:
graph = tg.TigerGraphConnection(host=hostName, graphname=graphName)

authToken = graph.getToken(secret)
authToken = authToken[0]
print(f"SHHHH.... Keep this a secret. Here's your token:\n {authToken}")

SHHHH.... Keep this a secret. Here's your token:
 o9ussc78scmn8qce8ilqahlt17an8dtq


In [6]:
show_schema = conn.gsql('''LS''')
print(show_schema)

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX Person(PRIMARY_ID id STRING, name STRING, email STRING, username STRING, created_at DATETIME) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Post(PRIMARY_ID id STRING, content STRING, posted_date DATETIME, deleted BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Hashtag(PRIMARY_ID tag STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Message(PRIMARY_ID id STRING, subject STRING, body STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE posted(FROM Person, TO Post, post_date DATETIME) WITH REVERSE_EDGE="reverse_posted"
- DIRECTED EDGE reverse_posted(FROM Post, TO Person, post_date DATETIME) WITH REVERSE_EDGE="posted"
- DIRECTED EDGE liked(FROM Person, TO Post, like_date DATETIME) WITH REVERSE_EDGE="reverse_liked"
- DIRECTED EDGE reverse_liked(FROM Post, TO Person, like_date DAT

In [7]:
results = conn.gsql('''
USE GRAPH test_graph11
''')
print(results)

Using graph 'test_graph11'


In [15]:
# DEFINE / CREATE ALL EDGES AND VERTICES
results = conn.gsql('''
  USE global
  CREATE VERTEX Account (PRIMARY_ID account_id int) WITH primary_id_as_attribute="true"
  CREATE VERTEX Transaction (PRIMARY_ID transaction_id int, from_acct int, to_acct int,trans_date DATETIME, fraud BOOL) WITH primary_id_as_attribute="true"
  CREATE VERTEX Alert (PRIMARY_ID alert_id int, account_id int, alert_type STRING) WITH primary_id_as_attribute="true"
  CREATE DIRECTED EDGE has_alert(From Account, To Alert) WITH REVERSE_EDGE="reverse_has_alert"
  CREATE DIRECTED EDGE sent_amount (From Account, To Transaction, to_account int, sent_date DATETIME) WITH REVERSE_EDGE="reverse_sent_money"
  CREATE DIRECTED EDGE received_amount (From Transaction, To Account, from_account int, receive_date DATETIME) WITH REVERSE_EDGE="reverse_received_money"
''')
print(results)

Successfully created vertex types: [Account].
Successfully created vertex types: [Transaction].
Successfully created vertex types: [Alert].
Successfully created edge types: [has_alert].
Successfully created reverse edge types: [reverse_has_alert].
Successfully created edge types: [sent_amount].
Successfully created reverse edge types: [reverse_sent_money].
Successfully created edge types: [received_amount].
Successfully created reverse edge types: [reverse_received_money].


In [16]:
results = conn.gsql('CREATE GRAPH test_graph11(Account, Transaction, Alert, has_alert, sent_amount, received_amount, reverse_sent_money, reverse_received_money, reverse_has_alert)')

In [18]:
conn.graphname="test_graph11"
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
conn = tg.TigerGraphConnection(host=hostName, graphname="test_graph11", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

10riue4cvinhp5b5j9mv8i8rmdht2ql7


In [25]:
show_schema = conn.gsql('''LS''')
print(show_schema)

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX Account(PRIMARY_ID account_id INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Transaction(PRIMARY_ID transaction_id INT, from_acct INT, to_acct INT, trans_date DATETIME, fraud BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Alert(PRIMARY_ID alert_id INT, account_id INT, alert_type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE has_alert(FROM Account, TO Alert) WITH REVERSE_EDGE="reverse_has_alert"
- DIRECTED EDGE reverse_has_alert(FROM Alert, TO Account) WITH REVERSE_EDGE="has_alert"
- DIRECTED EDGE sent_amount(FROM Account, TO Transaction, to_account INT, sent_date DATETIME) WITH REVERSE_EDGE="reverse_sent_money"
- DIRECTED EDGE reverse_sent_money(FROM Transaction, TO Account, to_account INT, sent_date DATETIME) WITH REVERSE_EDGE="sent_amount"
- DIRECTED EDGE received_amount(FROM Transaction, TO Ac

In [26]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['Account', 'Transaction', 'Alert']
Edges: ['send_money', 'receive_money', 'has_alert']


In [27]:

print(f"Results for Post vertex")
pprint(conn.getVertexType("Account"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("send_money"))


Results for Post vertex
{
  "Config": {
    "STATS": "OUTDEGREE_BY_EDGETYPE"
  },
  "IsLocal": true,
  "Attributes": [],
  "PrimaryId": {
    "AttributeType": {
      "Name": "INT"
    },
    "AttributeName": "account_id"
  },
  "Name": "Account"
}
-----------------
Results for liked edge
{
  "IsDirected": true,
  "ToVertexTypeName": "Transaction",
  "Config": {
    "REVERSE_EDGE": "reverse_send_money"
  },
  "IsLocal": true,
  "Attributes": [
    {
      "AttributeType": {
        "Name": "INT"
      },
      "AttributeName": "to_acct"
    },
    {
      "AttributeType": {
        "Name": "DATETIME"
      },
      "AttributeName": "sent_date"
    }
  ],
  "FromVertexTypeName": "Account",
  "Name": "send_money"
}


In [28]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 1446 Account vertices in the graph
There are 121457 Transaction vertices in the graph
There are 10 Alert vertices in the graph
--------------
Edge Counts
There are 0 send_money edges in the graph
There are 0 receive_money edges in the graph
There are 0 has_alert edges in the graph


In [32]:
results = conn.getVerticesById("Account", "424")
pprint(results)

[
  {
    "v_id": "424",
    "v_type": "Account",
    "attributes": {}
  }
]


In [34]:
df1 = conn.getVertexDataframe("Alert")
print(df1)

  v_id  account_id Alert_type
0    9        1140      cycle
1    5         103    fan_out
2    7         978      cycle
3    3         831    fan_out
4    6         128      cycle
5    1         666     fan_in
6    4         533    fan_out
7    0         554     fan_in
8    8        1303      cycle
9    2         265     fan_in
